In [1]:
import scanpy as sc
import pandas as pd
import numpy as np

Load anndata

In [2]:
adata = sc.read_h5ad("test.h5ad")

In [3]:
# # lognorm should be the default (expected)
# adata.X = adata.layers['logcounts']

Parameters

In [4]:
groupby='label'
de_method='wilcoxon'
resource_name = 'consensus'
complex_policy = 'min'
key_cols = ['source', 'target', 'ligand_complex', 'receptor_complex']
verbose = False

In [5]:
# only relevant if perms are done
n_perms = 1000
seed = 69

Run LIANA PIPE

In [6]:
from liana import liana_pipe

In [7]:
lr_res = liana_pipe(adata=adata, 
                    groupby=groupby, 
                    resource_name=resource_name,
                    de_method=de_method,
                    n_perms=n_perms,
                    seed=seed,
                    verbose=verbose,
                    _key_cols=key_cols,
                    resource=None,
                    use_raw=False,
                    layer='logcounts',
                   base=2)

/home/dbdimitrov/Repos/liana-py/liana/steady/liana_pipe.py:61: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['label'] = adata.obs[groupby]
/home/dbdimitrov/anaconda3/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)
/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value


In [8]:
lr_res.sort_values('ligand_means')

,ligand,ligand_complex,ligand_logfc,ligand_means,ligand_zscores,mat_mean,receptor,receptor_complex,receptor_logfc,receptor_means,receptor_zscores,source,target,ligand_means_sums,receptor_means_sums
2288,CSF1,CSF1,-0.025679,0.000000,-0.105409,0.199056,CSF2RA,CSF2RA,-0.031478,0.000000,-0.105409,CD8 T,CD8 T,0.035162,0.040544
2096,BTLA,BTLA,-0.027650,0.000000,-0.105409,0.199056,TNFRSF14,TNFRSF14,-0.340754,0.294117,-0.250196,CD8 T,CD8 T,0.037057,1.424242
2087,IL18BP,IL18BP,-0.072031,0.000000,-0.145695,0.199056,IL6R,IL6R,-0.037279,0.000000,-0.105409,CD8 T,CD8 T,0.087242,0.045404
2086,IL6,IL6,-0.152051,0.000000,-0.149471,0.199056,IL6ST,IL6R_IL6ST,-0.066984,0.000000,-0.146079,CD8 T,CD8 T,0.140459,0.083360
2085,IL6,IL6,-0.152051,0.000000,-0.149471,0.199056,IL6R,IL6R_IL6ST,-0.037279,0.000000,-0.105409,CD8 T,CD8 T,0.140459,0.045404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,B2M,B2M,0.429683,5.810702,0.728883,0.199056,CD247,CD247,-1.441419,0.269538,-0.764050,NK,B,16.490938,3.277351
4178,B2M,B2M,0.429683,5.810702,0.728883,0.199056,CD3G,CD3G,-0.119213,0.196046,-0.106673,NK,NK,16.490938,0.797412
4332,B2M,B2M,0.429683,5.810702,0.728883,0.199056,KLRC1,KLRC1,0.785538,0.642471,0.494169,NK,NK,16.490938,0.893647
2647,B2M,B2M,0.429683,5.810702,0.728883,0.199056,CD3D,CD3D,1.606492,2.190063,0.951525,NK,CD8 T,16.490938,3.070996


Tests

In [9]:
adata = sc.datasets.pbmc68k_reduced()

In [10]:
liana_pipe(adata=adata, 
                    groupby='bulk_labels', 
                    resource_name=resource_name,
                    de_method=de_method,
                    n_perms=n_perms,
                    seed=seed,
                    verbose=verbose,
                    _key_cols=key_cols,
                    use_raw=False,
                    layer=None,
                    resource=None)

/home/dbdimitrov/Repos/liana-py/liana/steady/liana_pipe.py:61: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['label'] = adata.obs[groupby]
/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value
/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:420: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(


,ligand,ligand_complex,ligand_logfc,ligand_means,ligand_zscores,mat_mean,receptor,receptor_complex,receptor_logfc,receptor_means,receptor_zscores,source,target,ligand_means_sums,receptor_means_sums
0,HLA-DRA,HLA-DRA,-2.180511,-0.880236,-0.874312,-0.000455,CD4,CD4,-1.642519,-0.017662,-0.019274,CD4+/CD25 T Reg,CD4+/CD25 T Reg,-5.165407,-0.577864
1,HLA-DQA2,HLA-DQA2,-2.168241,-0.753809,-0.745903,-0.000455,CD4,CD4,-1.642519,-0.017662,-0.019274,CD4+/CD25 T Reg,CD4+/CD25 T Reg,-4.698680,-0.577864
2,HLA-DQA1,HLA-DQA1,-2.134117,-0.750588,-0.740316,-0.000455,CD4,CD4,-1.642519,-0.017662,-0.019274,CD4+/CD25 T Reg,CD4+/CD25 T Reg,-4.792658,-0.577864
3,HLA-DRB1,HLA-DRB1,-2.217424,-0.884191,-0.874262,-0.000455,CD4,CD4,-1.642519,-0.017662,-0.019274,CD4+/CD25 T Reg,CD4+/CD25 T Reg,-5.469372,-0.577864
4,HLA-DQB1,HLA-DQB1,-2.218167,-0.707706,-0.692266,-0.000455,CD4,CD4,-1.642519,-0.017662,-0.019274,CD4+/CD25 T Reg,CD4+/CD25 T Reg,-4.407158,-0.577864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,TIMP1,TIMP1,-1.558552,0.200904,0.182052,-0.000455,CD63,CD63,-1.136314,0.231538,0.185384,Dendritic,Dendritic,-2.900203,-0.173462
4396,GNAI2,GNAI2,-0.103798,0.165708,0.165837,-0.000455,S1PR5,S1PR5,-7.255223,-0.080771,-0.103845,Dendritic,Dendritic,-1.459671,1.746308
4397,GNAI2,GNAI2,-0.103798,0.165708,0.165837,-0.000455,S1PR4,S1PR4,-2.830575,-0.460076,-0.436913,Dendritic,Dendritic,-1.459671,0.669178
4398,C1QA,C1QA,-0.987465,-0.157579,-0.156749,-0.000455,CD33,CD33,2.156790,0.683950,0.646014,Dendritic,Dendritic,-1.621117,-2.775528


Consensus

In [11]:
from liana.steady.scores.consensus import consensus

In [12]:
from functools import reduce
from scipy.stats import rankdata, beta

In [13]:
from liana.steady import cellphonedb, natmi, connectome, logfc, singlecellsignalr as sca

In [14]:
from liana.steady.aggregate import _robust_rank_aggregate, _rank_aggregate

In [15]:
_methods = [cellphonedb, connectome, logfc, natmi, sca]

In [18]:
adata = consensus(adata, groupby='bulk_labels',
                  use_raw=True
                  #, consensus_opts=False
                 )

/home/dbdimitrov/Repos/liana-py/liana/steady/liana_pipe.py:61: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['label'] = adata.obs[groupby]
/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value


CellPhoneDB
Connectome
log2FC
NATMI
SingleCellSignalR


In [19]:
lr_res = adata.uns['liana_res']

In [20]:
lr_res[lr_res['receptor_complex']=='CD8A_CD8B'].sort_values('spec_weight')

,source,target,ligand_complex,receptor_complex,lr_means,pvals,expr_prod,scaled_weight,lr_logfc,spec_weight,lrscore,ranked_magnitude,ranked_specificity,ranked_steady
2962,CD4+/CD25 T Reg,CD34+,LCK,CD8A_CD8B,1.000000,1.0,0.000000,0.436811,0.388214,0.000000,0.000000,0.746537,0.247577,0.478871
3340,Dendritic,CD34+,LCK,CD8A_CD8B,1.000000,1.0,0.000000,-0.395212,-1.045007,0.000000,0.000000,0.746537,1.000000,1.000000
988,CD8+ Cytotoxic T,CD4+/CD45RO+ Memory,LCK,CD8A_CD8B,1.000000,1.0,0.000000,0.235221,0.225348,0.000000,0.000000,0.746537,0.428207,0.806456
3130,CD8+/CD45RA+ Naive Cytotoxic,CD34+,LCK,CD8A_CD8B,1.000000,1.0,0.000000,0.336322,0.229000,0.000000,0.000000,0.746537,0.419641,0.791237
946,CD4+/CD45RO+ Memory,CD4+/CD45RO+ Memory,LCK,CD8A_CD8B,1.000000,1.0,0.000000,0.359003,0.337453,0.000000,0.000000,0.746537,0.298122,0.572018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1282,CD4+/CD25 T Reg,CD8+ Cytotoxic T,LCK,CD8A_CD8B,1.097794,0.0,0.917084,1.042482,1.323015,0.049596,0.616470,0.097997,0.000131,0.000012
1828,CD8+ Cytotoxic T,CD8+/CD45RA+ Naive Cytotoxic,LCK,CD8A_CD8B,1.029732,0.0,1.018060,1.087947,1.142487,0.055057,0.628741,0.121961,0.000131,0.000012
1786,CD4+/CD45RO+ Memory,CD8+/CD45RA+ Naive Cytotoxic,LCK,CD8A_CD8B,1.138704,0.0,1.197667,1.211728,1.254591,0.064771,0.647498,0.087112,0.000131,0.000012
1870,CD8+/CD45RA+ Naive Cytotoxic,CD8+/CD45RA+ Naive Cytotoxic,LCK,CD8A_CD8B,1.140837,0.0,1.201183,1.214151,1.266708,0.064961,0.647832,0.086329,0.000131,0.000012


## Each Method

CellPhoneDB

In [ ]:
from liana import cellphonedb

In [ ]:
%%time
adata = cellphonedb(adata, groupby='label', n_perms=1000)

In [ ]:
lr_res = adata.uns['liana_res']

In [ ]:
lr_res[lr_res['receptor_complex']=='CD8A_CD8B']

NATMI

In [ ]:
from liana import natmi

In [ ]:
adata = natmi(adata, groupby='label')

In [ ]:
adata.uns['liana_res']

SCA

In [ ]:
from liana import singlecellsignalr as sca

In [ ]:
adata = sca(adata, groupby='label')

In [ ]:
adata.uns['liana_res'].sort_values('lr_means', ascending=True)

Connectome

In [ ]:
from liana import connectome

In [ ]:
adata = connectome(adata, groupby='label')

In [ ]:
adata.uns['liana_res'].sort_values('scaled_weight', ascending=False)

logFC

In [ ]:
from liana import logfc

In [ ]:
adata = logfc(adata, groupby='label')

In [ ]:
adata.uns['liana_res'].sort_values('lr_logfc', ascending=False)

In [ ]:
np.exp(1)

try w/ real data

In [ ]:
adata = sc.datasets.pbmc3k_processed()
labels = adata.obs.louvain
cells = adata.obs_names

In [ ]:
adata = sc.datasets.pbmc3k()

In [ ]:
adata = adata[[x in cells for x in adata.obs_names]]

In [ ]:
adata.obs['label'] = labels

In [ ]:
sc.pp.log1p(adata)

In [ ]:
%%time
adata = cellphonedb(adata, groupby='label', n_perms=1000)

## LIANA PIPE LINE BY LINE

In [ ]:
# I get this from the score object
_complex_cols = ['ligand_means', 'receptor_means']
# change to full list and move to _var
_add_cols = ['ligand', 'receptor',
             'ligand_means_sums', 'receptor_means_sums',
             'ligand_zscores', 'receptor_zscores',
             'ligand_logfoldchanges', 'receptor_logfoldchanges',
             'ligand_logfc', 'receptor_logfc'
             'mat_mean',
             ]

_key_cols = ['source', 'target', 'ligand_complex', 'receptor_complex']

In [ ]:
from liana.utils.pre import check_adata, check_if_covered, format_vars, filter_resource
from liana.resource import select_resource
from liana.resource.select_resource import explode_complexes
from liana.steady.liana_pipe import _get_lr
from liana.utils.reassemble_complexes import reassemble_complexes
from scipy.sparse import csr_matrix
from liana.steady._permutations import get_means_perms

I need to double check if every step makes sense, as I build unit tests!!!

For example, are there any duplicated rows (source, target, ligand_complex, receptor_complex)

Double check again if correct subunit is kept, etc

In [ ]:
adata = check_adata(adata, True)

In [ ]:
# Define idents col name
adata.obs.label = adata.obs[groupby]

Load resource

In [ ]:
resource = select_resource(resource_name='consensus')

In [ ]:
# Decomplexify
resource = explode_complexes(resource)

In [ ]:
# Filter Resource
resource = filter_resource(resource, adata.var_names)

Create entities

In [ ]:
entities = np.union1d(np.unique(resource["ligand"]), np.unique(resource["receptor"]))

Check overlap between resource and adata

In [ ]:
check_if_covered(entities, adata.var_keys)

Get global mean for SCA before filtering

In [ ]:
if 'mat_mean' in _add_cols: # SHOULD BE METHOD NAME!
    adata.uns['mat_mean'] = np.mean(adata.X)

In [ ]:
# Filter to only include the relevant genes
adata = adata[:,np.intersect1d(entities, adata.var.index)]
adata

Get Stats

In [ ]:
lr_res = _get_lr(adata, resource, key_cols + _complex_cols + _add_cols, de_method)

In [ ]:
lr_res

Recomplexify

In [ ]:
# If I want to implement anything else but the min as complex policy, I would need to change this:
# temp = temp[[x=="INHBA_INHBB" for x in temp['ligand_complex']]].sort_values('receptor_complex')
# temp = temp[[x=="ACVR1B_ACVR2A" for x in temp['receptor_complex']]]

In [ ]:
lr_res = reassemble_complexes(lr_res, _key_cols, _complex_cols)

In [ ]:
lr_res

### SCORES

In [ ]:
from liana.steady.Method import Method, MethodMeta
from liana.steady.scores.cellphonedb import _cpdb_score

CELLPHONEDB

In [ ]:
_cellphonedb = MethodMeta(method_name = "cellphonedb",
                          complex_cols = ['ligand_means', 'receptor_means'], 
                          add_cols = ['ligand', 'receptor'],
                          fun = _cpdb_score,
                          magnitude = 'lr_means',
                          specificity = 'pvals',
                          permute = True,
                          reference = 'Efremova et al., 2020')

In [ ]:
cellphonedb = Method(_SCORE=_cellphonedb)

In [ ]:
adata = cellphonedb(adata, groupby, seed=666)

In [ ]:
adata.uns['liana_res']

logFC re-implement

Magnitude as None

In [ ]:
lr_res['logfc'] = lr_res[['ligand_logfoldchanges', 'receptor_logfoldchanges']].mean(1)

In [ ]:
lr_res.sort_values(by='logfc', key=abs, ascending=False)

Re-implement Connectome

Both Specificity and Magnitude

In [ ]:
lr_res['edge_weight'] = lr_res[['ligand_zscores', 'receptor_zscores']].mean(1)